In [11]:
import pandas as pd
import os

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df.head()

,Days_Since_Start,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Volume,TSLA_RSI_7,TSLA_RSI_14,TSLA_MACD,TSLA_MACD_Signal,...,NEE_MACD_Signal,HD_Open,HD_High,HD_Low,HD_Close,HD_Volume,HD_RSI_7,HD_RSI_14,HD_MACD,HD_MACD_Signal
0,196,1.424667,1.483333,1.394667,1.460667,37297500.0,92.220196,43.408794,-0.064685,-0.083391,...,0.059857,19.278965,19.349948,19.058918,19.215080,17061600,25.229445,25.954283,-0.802271,-0.824908
1,197,1.456667,1.456667,1.336667,1.353333,27379500.0,71.260949,39.206237,-0.058209,-0.078355,...,0.071355,18.987938,19.896520,18.895661,19.825537,23279400,44.368562,43.190715,-0.756203,-0.811167
2,198,1.377333,1.393333,1.300000,1.348000,18787500.0,74.198423,39.159154,-0.052897,-0.073263,...,0.079636,19.719066,19.882326,19.413839,19.499018,17161300,37.425292,44.047680,-0.737539,-0.796441
3,199,1.366667,1.416667,1.358000,1.400000,14367000.0,72.916658,46.915168,-0.043984,-0.067407,...,0.087182,19.669378,20.208849,19.669378,20.031391,26142700,44.414890,52.659520,-0.672043,-0.771562
4,200,1.412667,1.437333,1.404000,1.419333,9804000.0,65.010249,57.983173,-0.034957,-0.060917,...,0.093594,19.974605,20.130768,19.804246,20.052687,18979200,49.562788,54.446405,-0.611372,-0.739524


In [12]:
import pandas as pd

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Days_Since_Start'])
df.set_index('Days_Since_Start', inplace=True)

# 确保其他列是数值类型，去掉可能的字符串类型列（例如日期）
df = df.apply(pd.to_numeric, errors='coerce')

# 检查数据的头部
print(df.head())

                  TSLA_Open  TSLA_High  TSLA_Low  TSLA_Close  TSLA_Volume  \
Days_Since_Start                                                            
196                1.424667   1.483333  1.394667    1.460667   37297500.0   
197                1.456667   1.456667  1.336667    1.353333   27379500.0   
198                1.377333   1.393333  1.300000    1.348000   18787500.0   
199                1.366667   1.416667  1.358000    1.400000   14367000.0   
200                1.412667   1.437333  1.404000    1.419333    9804000.0   

                  TSLA_RSI_7  TSLA_RSI_14  TSLA_MACD  TSLA_MACD_Signal  \
Days_Since_Start                                                         
196                92.220196    43.408794  -0.064685         -0.083391   
197                71.260949    39.206237  -0.058209         -0.078355   
198                74.198423    39.159154  -0.052897         -0.073263   
199                72.916658    46.915168  -0.043984         -0.067407   
200             

In [13]:
rows, cols = df.shape
print(f"DataFrame 共有 {rows} 行, {cols} 列")


DataFrame 共有 2633 行, 91 列


In [14]:
# 检查数据的列名
print(df.columns)

# 检查是否有缺失值
print(df.isnull().sum())


Index(['TSLA_Open', 'TSLA_High', 'TSLA_Low', 'TSLA_Close', 'TSLA_Volume',
       'TSLA_RSI_7', 'TSLA_RSI_14', 'TSLA_MACD', 'TSLA_MACD_Signal',
       'AAPL_Open', 'AAPL_High', 'AAPL_Low', 'AAPL_Close', 'AAPL_Volume',
       'AAPL_RSI_7', 'AAPL_RSI_14', 'AAPL_MACD', 'AAPL_MACD_Signal', 'GE_Open',
       'GE_High', 'GE_Low', 'GE_Close', 'GE_Volume', 'GE_RSI_7', 'GE_RSI_14',
       'GE_MACD', 'GE_MACD_Signal', 'QQQ_Open', 'QQQ_High', 'QQQ_Low',
       'QQQ_Close', 'QQQ_Volume', 'QQQ_RSI_7', 'QQQ_RSI_14', 'QQQ_MACD',
       'QQQ_MACD_Signal', 'NVDA_Open', 'NVDA_High', 'NVDA_Low', 'NVDA_Close',
       'NVDA_Volume', 'NVDA_RSI_7', 'NVDA_RSI_14', 'NVDA_MACD',
       'NVDA_MACD_Signal', 'UNH_Open', 'UNH_High', 'UNH_Low', 'UNH_Close',
       'UNH_Volume', 'UNH_RSI_7', 'UNH_RSI_14', 'UNH_MACD', 'UNH_MACD_Signal',
       'CAT_Open', 'CAT_High', 'CAT_Low', 'CAT_Close', 'CAT_Volume',
       'CAT_RSI_7', 'CAT_RSI_14', 'CAT_MACD', 'CAT_MACD_Signal', 'AMZN_Open',
       'AMZN_High', 'AMZN_Low', 'AMZN_

In [15]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd

class TradingEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}
    
    def __init__(self, df, window_size=10, initial_balance=10000, max_drawdown=0.3, risk_free_rate=0.01, transaction_cost=0.0005):
        super(TradingEnv, self).__init__()
        
        self.df = df
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.max_drawdown = max_drawdown
        self.risk_free_rate = risk_free_rate
        self.transaction_cost = transaction_cost  # 交易成本降低到 0.05%
        
        # 获取股票代码列表，排除 "Days_Since_Start"
        self.tickers = sorted(set(col.split('_')[0] for col in df.columns if '_' in col and not col.startswith("Days")))
        self.num_stocks = len(self.tickers)
        self.features_per_stock = 9  # 9个特征 (Open, High, Low, Close, Volume, RSI_7, RSI_14, MACD, MACD_Signal)
        
        # 定义状态空间 (window_size 天 * 10 只股票 * 9 特征)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size * 90,), dtype=np.float32)

        # 动作空间 (-0.5 ~ 0.5 代表买入 / 卖出比例，减少剧烈变动)
        self.action_space = spaces.Box(low=-0.5, high=0.5, shape=(self.num_stocks,), dtype=np.float32)

        self.reset()
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.shares_held = np.zeros(self.num_stocks)  # 每只股票的持仓
        self.total_profit = 0
        self.peak_value = self.initial_balance
        self.done = False
        self.history = [self.initial_balance]  # 记录每一步的资产值

        return self._next_observation(), {}
    
    def _next_observation(self):
        feature_cols = [col for col in self.df.columns if col != "Days_Since_Start"]
        obs = self.df.iloc[self.current_step - self.window_size:self.current_step][feature_cols].values.flatten()
        return obs

    def step(self, action):
        if self.done:
            return self._next_observation(), 0, True, False, {}

        close_prices = self.df.iloc[self.current_step, [self.df.columns.get_loc(f"{t}_Close") for t in self.tickers]]
        
        # 计算当前总资产
        total_value = self.balance + np.dot(self.shares_held, close_prices.values)
        
        # 计算目标持仓
        target_values = np.clip(action, -0.5, 0.5) * total_value  # 限制变动幅度
        target_shares = target_values / (close_prices.values + 1e-6)  # 避免除零错误
        delta_shares = target_shares - self.shares_held

        # 执行交易
        for i in range(self.num_stocks):
            current_price = close_prices.iloc[i]
            cost = abs(delta_shares[i]) * current_price * self.transaction_cost  

            if delta_shares[i] > 0:  # 买入
                max_shares = self.balance // (current_price + cost)
                buy_shares = min(max_shares, delta_shares[i])  
                self.shares_held[i] += buy_shares
                self.balance -= buy_shares * current_price + cost

            elif delta_shares[i] < 0:  # 卖出
                sell_shares = min(abs(delta_shares[i]), self.shares_held[i])  
                self.shares_held[i] -= sell_shares
                self.balance += sell_shares * current_price - cost

        # 计算新资产总值
        total_value = self.balance + np.dot(self.shares_held, close_prices.values)
        prev_total_value = self.history[-1] if len(self.history) > 0 else self.initial_balance
        daily_return = (total_value - prev_total_value) / (prev_total_value + 1e-6)

        self.total_profit = total_value - self.initial_balance

        # 计算回撤
        self.peak_value = max(self.peak_value, total_value)
        drawdown = (self.peak_value - total_value) / (self.peak_value + 1e-6)

        # 计算夏普比率
        self.history.append(total_value)
        sharpe_ratio = 0  # 默认值

        if len(self.history) > 30:
            returns = np.diff(self.history[-30:]) / (np.array(self.history[-30:-1]) + 1e-6)
            if len(returns) > 1 and np.std(returns) > 0:
                sharpe_ratio = (np.mean(returns) - self.risk_free_rate) / (np.std(returns) + 1e-6)

        # 计算奖励
        reward = daily_return * 100 - drawdown * 2 + sharpe_ratio * 0.2  # 调整奖励函数
        
        # 如果回撤超过 30%，给予额外惩罚但不终止
        if drawdown >= self.max_drawdown:
            reward -= 10

        # 更新状态
        self.current_step += 1
        self.done = self.current_step >= len(self.df) - 1  # 只在数据用完时终止

        return self._next_observation(), reward, self.done, False, {}
    
    def render(self, mode="human"):
        print(f'Step: {self.current_step}, Balance: {self.balance:.2f}, Shares: {self.shares_held}, Profit: {self.total_profit:.2f}')

print("TradingEnv ready!")


TradingEnv ready!


In [16]:
import gymnasium as gym
import numpy as np

# 假设 df 是你已经加载并预处理好的 DataFrame，包含所有需要的特征
# df = pd.read_csv('your_stock_data.csv')

# 创建环境实例
env = TradingEnv(df)  # 使用你的数据

# 测试环境初始化
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

# 测试 reset 和 _next_observation
obs, info = env.reset()
print("Initial Observation:", obs)


Observation Space: Box(-inf, inf, (900,), float32)
Action Space: Box(-0.5, 0.5, (10,), float32)
Initial Observation: [ 1.42466700e+00  1.48333299e+00  1.39466703e+00  1.46066701e+00
  3.72975000e+07  9.22201958e+01  4.34087939e+01 -6.46850475e-02
 -8.33913485e-02  7.51978817e+00  7.51978817e+00  7.21042614e+00
  7.39038515e+00  1.02447800e+09  1.41132493e+01  2.77689318e+01
 -6.74227941e-02 -2.62855064e-02  5.17271744e+01  5.20804997e+01
  5.11265131e+01  5.16565094e+01  1.02396750e+07  4.15999081e+01
  4.42771895e+01 -9.42785420e-01 -1.24506051e+00  3.90346988e+01
  3.93331427e+01  3.87099192e+01  3.92541428e+01  8.18008000e+07
  5.85526974e+01  4.73683933e+01 -1.46861507e-01 -2.96142639e-01
  2.33627513e-01  2.40734902e-01  2.32022593e-01  2.39817828e-01
  7.98440000e+08  5.35545959e+01  4.24170542e+01 -1.02271605e-02
 -1.21284987e-02  2.44886278e+01  2.48742751e+01  2.43359767e+01
  2.47617950e+01  8.57550000e+06  7.56758722e+01  6.51206712e+01
  9.71865263e-02 -5.71588427e-02  4.45

In [17]:
# 测试执行一步操作
action = np.random.uniform(-1, 1, size=(env.num_stocks,))  # 随机生成动作
print("Test Action:", action)

# 进行一步操作并获取返回的观察值、奖励等
obs, reward, done, truncated, info = env.step(action)
print("New Observation:", obs)
print("Reward:", reward)
print("Done:", done)


Test Action: [-0.21664514  0.30789132  0.47769793  0.86631897  0.00354638  0.30574157
  0.87594142  0.25397202  0.08021628  0.01884853]
New Observation: [ 1.45666695e+00  1.45666695e+00  1.33666694e+00  1.35333300e+00
  2.73795000e+07  7.12609494e+01  3.92062370e+01 -5.82086295e-02
 -7.83548047e-02  7.30973612e+00  7.61067180e+00  7.22276563e+00
  7.58027697e+00  1.07495080e+09  3.25979834e+01  4.52739356e+01
 -6.54673241e-02 -3.41218700e-02  5.07378612e+01  5.30344895e+01
  5.07378612e+01  5.27871590e+01  1.40386600e+07  4.96553316e+01
  5.73718569e+01 -8.47155793e-01 -1.16547957e+00  3.86221417e+01
  3.97281380e+01  3.84992537e+01  3.97281380e+01  8.90612000e+07
  6.01264421e+01  6.52173061e+01 -9.48661735e-02 -2.55887346e-01
  2.33627499e-01  2.47154484e-01  2.31564043e-01  2.45778874e-01
  1.16098800e+09  5.79399830e+01  5.31089041e+01 -9.26364943e-03
 -1.15555288e-02  2.51876161e+01  2.53000962e+01  2.42958053e+01
  2.47698307e+01  1.34939000e+07  7.21651289e+01  7.27274288e+01
  

In [18]:
# 测试多步操作
for i in range(10):
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Step {i+1}, Reward: {reward}, Total Profit: {env.total_profit}, Done: {done}")
    if done:
        break


Step 1, Reward: -0.07066069707122988, Total Profit: -19.98036389760091, Done: False
Step 2, Reward: 0.39637235038334695, Total Profit: 19.57767450670144, Done: False
Step 3, Reward: -1.215876686034583, Total Profit: -99.85929509683956, Done: False
Step 4, Reward: -1.1994185540964233, Total Profit: -213.98779392426513, Done: False
Step 5, Reward: 0.06473611181176125, Total Profit: -203.2990841459432, Done: False
Step 6, Reward: -0.5340333300319648, Total Profit: -250.33848998469875, Done: False
Step 7, Reward: -0.687286844715796, Total Profit: -310.9148425543117, Done: False
Step 8, Reward: -0.3515276978744561, Total Profit: -338.0578793330442, Done: False
Step 9, Reward: 0.28291782905709756, Total Profit: -304.4728431644835, Done: False
Step 10, Reward: 0.6056559734675088, Total Profit: -240.71383908091593, Done: False


In [19]:
# 测试环境是否能正常结束
env.reset()
while not env.done:
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Balance: {env.balance}, Total Profit: {env.total_profit}")


Balance: 19.540372540757772, Total Profit: -18.888527798337236
Balance: 4375.837341629913, Total Profit: -53.566824793542764
Balance: 1226.890018073542, Total Profit: -10.661385665111084
Balance: 21.4536813189483, Total Profit: -53.45028120926145
Balance: 1625.8469698789686, Total Profit: -103.45849410173287
Balance: 62.85124289605528, Total Profit: -80.2772056967915
Balance: 1782.9497359086042, Total Profit: -140.26435701193077
Balance: 4329.503923442535, Total Profit: -404.5333552780903
Balance: 27.63251614392606, Total Profit: -395.2613030719385
Balance: 4802.495331676989, Total Profit: -454.20888316413766
Balance: 24.149138558324537, Total Profit: -444.1802250572582
Balance: 3047.148589242267, Total Profit: -326.7892562324705
Balance: 3248.335524515751, Total Profit: -324.39265529668046
Balance: -3.311577326678652, Total Profit: -300.419418416046
Balance: 8.640497045182805, Total Profit: -282.4692799493114
Balance: 8910.441857058859, Total Profit: -332.45143544880193
Balance: 47.15

In [20]:
import gymnasium as gym
import numpy as np
import pandas as pd
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.evaluation import evaluate_policy


file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
# 加载并准备数据
df = pd.read_csv(file_path)  # 加载您的股票数据
env = TradingEnv(df)  # 使用数据创建环境

# 将环境封装成 VecEnv 格式，SAC 需要该格式来进行训练
env = DummyVecEnv([lambda: env])

# 定义评估回调
eval_callback = EvalCallback(
    env,
    best_model_save_path="./sac_model/",
    log_path="./logs/",
    eval_freq=5000,  # 每5000步评估一次
    deterministic=True,
    render=False
)

# 定义保存模型的回调
checkpoint_callback = CheckpointCallback(
    save_freq=5000,  # 每5000步保存一次
    save_path="./checkpoints/",
    name_prefix="sac_model"
)

# 创建并训练 SAC 模型，调整一些参数
model = SAC("MlpPolicy", env, verbose=1, 
            buffer_size=500000,  # 减小缓冲区大小
            learning_rate=0.0001,  # 调低学习率
            gamma=0.99, tau=0.005,  # 保持 tau 不变，但可以根据需要调整
            ent_coef='auto',  # 使用自动调整的熵系数
            policy_kwargs=dict(net_arch=[256, 256]),  # 更大的网络可以考虑增加层数
            tensorboard_log="./logs/"
            )

# 开始训练
model.learn(total_timesteps=1000000, callback=[eval_callback, checkpoint_callback])

# 保存模型
model.save("sac_trained_model")

# 评估模型
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")


Using cuda device
Logging to ./logs/SAC_3


/home/jesse/anaconda3/envs/rl_py/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=5000, episode_reward=-22287.05 +/- 0.00
Episode length: 2622.00 +/- 0.00
----------------------------------
| eval/              |           |
|    mean_ep_length  | 2.62e+03  |
|    mean_reward     | -2.23e+04 |
| time/              |           |
|    total_timesteps | 5000      |
| train/             |           |
|    actor_loss      | 3.35e+06  |
|    critic_loss     | 6.34e+10  |
|    ent_coef        | 1.63      |
|    ent_coef_loss   | -109      |
|    learning_rate   | 0.0001    |
|    n_updates       | 4899      |
----------------------------------
New best mean reward!


/home/jesse/anaconda3/envs/rl_py/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=-22287.05 +/- 0.00
Episode length: 2622.00 +/- 0.00
----------------------------------
| eval/              |           |
|    mean_ep_length  | 2.62e+03  |
|    mean_reward     | -2.23e+04 |
| time/              |           |
|    total_timesteps | 10000     |
| train/             |           |
|    actor_loss      | 3.22e+06  |
|    critic_loss     | 3.48e+10  |
|    ent_coef        | 2.69      |
|    ent_coef_loss   | -221      |
|    learning_rate   | 0.0001    |
|    n_updates       | 9899      |
----------------------------------


...

/home/jesse/anaconda3/envs/rl_py/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=280000, episode_reward=-22287.05 +/- 0.00
Episode length: 2622.00 +/- 0.00
----------------------------------
| eval/              |           |
|    mean_ep_length  | 2.62e+03  |
|    mean_reward     | -2.23e+04 |
| time/              |           |
|    total_timesteps | 280000    |
| train/             |           |
|    actor_loss      | 3.25e+14  |
|    critic_loss     | 1.01e+29  |
|    ent_coef        | 1.36e+12  |
|    ent_coef_loss   | -6.29e+03 |
|    learning_rate   | 0.0001    |
|    n_updates       | 279899    |
----------------------------------


/home/jesse/anaconda3/envs/rl_py/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=285000, episode_reward=-22287.05 +/- 0.00
Episode length: 2622.00 +/- 0.00
----------------------------------
| eval/              |           |
|    mean_ep_length  | 2.62e+03  |
|    mean_reward     | -2.23e+04 |
| time/              |           |
|    total_timesteps | 285000    |
| train/             |           |
|    actor_loss      | 5.31e+14  |
|    critic_loss     | 2.77e+29  |
|    ent_coef        | 2.24e+12  |
|    ent_coef_loss   | -6.39e+03 |
|    learning_rate   | 0.0001    |
|    n_updates       | 284899    |
----------------------------------


KeyboardInterrupt: 